In [ ]:
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
from snowflake.snowpark.functions import col

In [ ]:
# Import python packages
#%pip install <tabulate>
import streamlit as st
import pandas as pd
import altair as alt
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
SELECT rc.NAME as RACE,rc.YEAR,rc.ROUND,rc.CIRCUITID,C.NAME AS CONSTRUCTOR,CONCAT(D.FORENAME,' ',D.SURNAME) AS DRIVER,RS.LAPS,S.STATUS,RS.POSITION,RS.MILLISECONDS,RS.POINTS
FROM PLAYGROUND.DATATHON.RACES rc
left join playground.datathon.RESULTS rs on rs.raceid = rc.raceid
left join playground.datathon.status s on rs.statusid = s.statusid
left join playground.datathon.constructors c on rs.constructorid = c.constructorid
left join playground.datathon.drivers d on rs.driverid = d.driverid

In [ ]:
dg5_df1 = cell3.to_pandas()

#Missing values 
missing = dg5_df1.isnull().sum()
print("Missing Values :\n", missing)

In [ ]:
#Cleanse the data by removing the missing values
#dg5_df1 = cell3.to_pandas()
dg5_df1 = dg5_df1.dropna(subset=["POSITION","MILLISECONDS"])

#table=tabulate(dg5_df1.values,headers=dg5_df1.columns,tablefmt='grid')
#output_file = "C:\\Users\\tanu.d.gupta\\clean_races.csv"
#dg5_df1.to_csv(output_file, index=False)
#print(f"Query output saved to {output_file}")
#print(table)


In [ ]:
# Verify Missing values 
missing1 = dg5_df1.isnull().sum()
print("Missing Values :/n", missing1)

In [ ]:
dg5_df1.info()

In [ ]:
dg5_df1.describe()

In [ ]:
#Average points by Constructor
avg_pts_constructor = dg5_df1.groupby("CONSTRUCTOR")["POINTS"].mean().sort_values(ascending=False)
print("Avg points of a constructor:\n", avg_pts_constructor)

In [ ]:
#Average Laps by Driver
total_points_driver = dg5_df1.groupby("DRIVER")["POINTS"].sum().sort_values(ascending=False)
print("Total Points by a Driver:\n", total_points_driver)

In [ ]:
#Bar chart for averagee points
avg_pts_constructor.head(10).plot(kind="bar", figsize=(10, 5), title="Top Constructors by Average points", ylabel="POINTS")
plt.grid(True)
plt.show()

In [ ]:
#Bar chart for Top Drivers
total_points_driver.head(10).plot(kind="bar", figsize=(10, 5), title="Top Drivers securing highest points", ylabel="POINTS")
plt.grid(True)
plt.show()

In [ ]:
dg5_df1['MILLISECONDS'].median()

In [ ]:
dg5_df1['MILLISECONDS'].std()

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(dg5_df1['MILLISECONDS'].dropna())

In [ ]:

# Histograms (this gives us a quick diagnostic of data—such as whether values are skewed, normally distributed, or have outliers.)
dg5_df1.hist(bins=20, figsize=(10, 6))
plt.suptitle("Feature Distributions")
plt.tight_layout()
plt.show()

In [ ]:
# Histograms (this gives us a quick diagnostic of data—such as whether values are skewed, normally distributed, or have outliers.)
plt.figure(figsize=(10, 6))
sns.histplot(dg5_df1['MILLISECONDS'].dropna(), bins=20, kde=True, color='purple')
plt.title("Lap time Distributions")
plt.xlabel('Lap time (MS)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
# Box plot for constructor comparision
plt.figure(figsize=(12, 6))
sns.boxplot(x='CONSTRUCTOR', y='MILLISECONDS', data=dg5_df1, hue='CONSTRUCTOR', palette='Set2', legend=False)
plt.xticks(rotation=45)
plt.title("Lap time Distributions by COnstructors")
plt.ylabel('Lap time (MS)')
plt.show()

In [ ]:
#Top constructors vs Lap time

top_20 = dg5_df1['CONSTRUCTOR'].value_counts().nlargest(20).index
df_top_20 = dg5_df1[dg5_df1['CONSTRUCTOR'].isin(top_20)]

plt.figure(figsize=(12, 6))
sns.boxplot(
    data=df_top_20,
    x='CONSTRUCTOR',
    y='MILLISECONDS',
    hue='CONSTRUCTOR',
    palette='Set2',
    legend=False
)
plt.xticks(rotation=45)
plt.title('Top 20 Constructors vs. Lap Time (Milliseconds)')
plt.tight_layout()
plt.show()

In [ ]:
#Lap time over Race laps
#This will help in analysing performance consistency across the race

dg5_df1['LAPS']=pd.to_numeric(dg5_df1['LAPS'], errors='coerce')

df_filtered = dg5_df1[dg5_df1['CONSTRUCTOR'].isin(['McLaren','Mercedes'])]

plt.figure(figsize=(14, 6))
sns.lineplot(
    data=df_filtered,
    x='LAPS',
    y='MILLISECONDS',
    hue='CONSTRUCTOR',
    estimator='mean',
    errorbar=None,
    palette='Set2',
    legend=False
)
plt.title('Average Lap Time per Lap : McLaren vs Mercedes')
plt.xlabel('Lap Number')
plt.ylabel('Lap Time (ms)')
plt.grid(True)
plt.tight_layout()
plt.show()


Feature Engineering

In [ ]:
#Create new columns
#Avg_speed from millisec and lap distance
#Assume each lap is 5.8km

df_fe_clean = dg5_df1[dg5_df1['MILLISECONDS'] > 0]

#Create new column avg_speed_kmph
df_fe_clean['AVG_SPEED_KMPH'] = (5.8 / (df_fe_clean['MILLISECONDS'] / 1000)) * 3600
df_fe_clean[['LAPS','CONSTRUCTOR','MILLISECONDS', 'AVG_SPEED_KMPH']].head(10)


In [ ]:
#Encode categorical variables
# One-Hot Encode Constructor name (for ML models)
df_encoded = pd.get_dummies(df_fe_clean, columns=['CONSTRUCTOR'], prefix='make')
df_encoded.head()

In [ ]:
#Identify top 20 constructors
top_makes = df_fe_clean['CONSTRUCTOR'].value_counts().nlargest(20).index

#Create a mask column to idenstidy top makes and categorize rest as others
df_fe_clean['CONSTRUCTOR_TOP'] = df_fe_clean['CONSTRUCTOR'].apply(
    lambda x: x if x in top_makes else 'Other'
)

#
df_encoded = pd.get_dummies(df_fe_clean, columns=['CONSTRUCTOR_TOP'], prefix='make', drop_first=True)


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
dg5_df1['Constructor_encoded'] = le.fit_transform(dg5_df1['CONSTRUCTOR'].astype(str))
dg5_df1[['CONSTRUCTOR', 'Constructor_encoded']].drop_duplicates().head(10)

In [ ]:
#Handle missing values

dg5_df1.isnull().sum()

dg5_df1['MILLISECONDS'] = dg5_df1['MILLISECONDS'].fillna(dg5_df1['MILLISECONDS'].median())
dg5_df1['AVG_SPEED_KMPH'] = df_fe_clean['AVG_SPEED_KMPH'].fillna(df_fe_clean['AVG_SPEED_KMPH'].median())


In [ ]:
#Feature correlation

plt.figure(figsize=(10,6))
sns.heatmap(dg5_df1[['MILLISECONDS', 'AVG_SPEED_KMPH', 'POINTS']].corr(), annot=True, cmap='coolwarm')
plt.title('Feature correlation matrix')
plt.show()

In [ ]:
x = dg5_df1[['CONSTRUCTOR','DRIVER','LAPS']]
y= dg5_df1['MILLISECONDS']

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

#Categorical and numerical columns
cat_features = ['CONSTRUCTOR','DRIVER']
num_features = ['LAPS']

#ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore').cat_features),
        ('num', StandardScaler(),num_features)
    ]
)


In [ ]:
#Create the pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('regressor',LinearRegression())
])

In [ ]:
#Train the pipeline

model_pipleline.fit(X,y)

In [ ]:
# Make predictions
preds = model_pipeline.predict(X.head())
print("Predicted lap time",preds)

In [ ]:
#Evaluate Model (r2)
from sklearn.metrics import r2_score

r2 = r2_score(y, model_pipeline.predict(X))
print(f"R2 score: {r2:.3f}")

In [ ]:
select * from playground.datathon.circuits